<div class="alert alert-info">
    This is a tutorial on integrating sql to do data analysis in python. Some of the cells bellow are hiddened ( shown as <code>...</code>) and you should try to google the answer first before unhiding them.
</div>

# Typical workflow for a data analysis problem in Python

General procedure:

* Breakdown your problem into components
* For each component, identify the input and the required output
* Find a good library and the corresponding function to consume the input and produce the output
* Sometimes, it is good to spend time to understand the mathematics, the algorithms / data structure, and the design behind the library
* Write a bit of Python to prepare the data, pipe the components, add small utilities, improve code reusability, etc.

In principle, unless it is super easy, always look for existing libraries before attempting to implement it yourself. You are not writing c++ :)

## An example problem
We have bibliometric data in our MSSQL server. Let's see how well we can predict the citations of publications from other variables.

### Components
* Get the data from MSSQL server to python
* Regression analysis
* Visualization

#### Let's google: connect to MSSQL server from python

* pyodbc: https://docs.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver15
* More than one choice: https://stackoverflow.com/questions/57063181/what-are-the-advantages-and-disadvantages-of-using-pypyodbc-pymssql-and-pyodbc
* You know you are going to use pandas anyway, and pandas doc suggests sqlalchemy: https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html
* sqlalchemy vs pyodbc: https://www.quora.com/What-is-the-difference-between-PyODBC-and-SQLAlchemy
* Execute sql query directly in jupyter notebook: https://towardsdatascience.com/how-to-run-sql-queries-from-a-jupyter-notebook-aaa18e59e7bc

#### Create sql engine to load data into pandas dataframe

Import libraries / functions

You should have installed `pyodbc` as the backend

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

How to connect to mssql: https://docs.sqlalchemy.org/en/14/core/engines.html

Create sql engine by sqlalchemy, change `laikh` to your username

In [ ]:
engine = create_engine("mssql+pyodbc://laikh@P-CWTS-010260", fast_executemany=True)

It is good to briefly see how the database looks like before getting the data into pandas

This is a magic provided by `ipython-sql`, remember to change `laikh` to your username

In [ ]:
%reload_ext sql
%sql mssql+pyodbc://laikh@P-CWTS-010260

`%%sql` indicates that this cell should be excuted as sql instead of python

List all databases in the server

In [ ]:
%%sql
SELECT name 
FROM master.sys.databases

View the top rows from the wos_2013.dbo.pub database

In [ ]:
%%sql
SELECT TOP (5) *
FROM wos_2113.dbo.pub

Execute a sql query to load the publication data of a specific cluster into a pandas dataframe

In [ ]:
sql_query = """
SELECT *
FROM wos_2113..pub
INNER JOIN wos_2113_classification..clustering AS c
ON c.ut = pub.ut
WHERE cluster_id1 = 116
"""

In [ ]:
df = pd.read_sql(sql_query, con=engine)

#### Visualizing the data
Before doing any analysis, we should try to understand the features of the data by visualization

Visualize the number of publication by year, using the simple `plot()` function for pandas

In [ ]:
df.groupby(['pub_year']).size().plot()

There are many good visualization libaries in Python

Here, we use the well-known `seaborn` library, it integrates well with pandas dataframe. You may try to google for other alternatives

In [ ]:
import seaborn as sns

Plot the number of citations versus the year of publication, also visualize the effect of open access: https://seaborn.pydata.org/generated/seaborn.lineplot.html

In [ ]:
sns.lineplot(data=df, x="pub_year", y="n_cits", hue="is_open_access")

The labels in x-axis are strange, year should be integer.

Therefore, I googled to find ways to turn the labels to integer, and I copied the code from https://stackoverflow.com/questions/30327153/seaborn-pylab-changing-xticks-from-float-to-int?rq=1

In [ ]:
from matplotlib.ticker import FuncFormatter

In [ ]:
ax = sns.lineplot(data=df, x="pub_year", y="n_cits", hue="is_open_access")
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
ax

It still doesn't look correct, the intervals are not even, so I used different keywords to find this: https://stackoverflow.com/questions/30914462/matplotlib-how-to-force-integer-tick-labels

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
ax = sns.lineplot(data=df, x="pub_year", y="n_cits", hue="is_open_access")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax

#### Prepare for data analysis: transforming the data / feature engineering

You can simply use the data in its original form for regression analysis, but it is often beneficial to transform the data to more representative features based on your knowledge.

Here, we want to transform the data to the number of years since the year of publication.

For such a simple arithmetic, pandas support the `-` operation.

In [ ]:
df["num_year_1"] = 2021 - df["pub_year"]

In [ ]:
df["num_year_1"].head()

You may also use the `map` function to achieve exactly the same thing.

The `lambda x:` declares a function to transform each element in the series (`df["pub_year"]` is a series) to a new element. Since it can be any single variable function, it is more flexible than the `-` operator.

In [ ]:
df["num_year_2"] = df["pub_year"].map(lambda x: 2021 - x)  

In [ ]:
df["num_year_2"].head()

You may want your `lambda` function to act on the whole row in the dataframe, such that you can take values in other columns in the function. You can use `apply` in this case.

In [ ]:
df["num_year_3"] = df.apply(lambda x: 2021 - x["pub_year"], axis=1)

You may prefer defining a function outside of the `lambda` function for better layout

In [ ]:
def compute_num_year(x):
    return 2021 - x["pub_year"]
df["num_year_4"] = df.apply(lambda x: compute_num_year(x), axis=1)

As you can see, all the 4 methods produce the same result

In [ ]:
df[["num_year_1", "num_year_2", "num_year_3",  "num_year_4"]].head()

The data order is not random, we should shuffle the dataframe, I found this by google: https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

Now, it is random.

In [ ]:
df[["num_year_1", "num_year_2", "num_year_3",  "num_year_4"]].head()

There are always multiple ways to achieve the same goal in Python, one could be "better" than the other in terms of performance, but you should always ensure things work first before considering alternative "better" methods.

#### Let's google: regression library in python

* scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* statmodels: https://www.statsmodels.org/stable/regression.html
* xgboost: https://xgboost.readthedocs.io/en/stable/python/python_intro.html
* xgboost vs scikit-learn: https://stats.stackexchange.com/questions/282459/xgboost-vs-python-sklearn-gradient-boosted-trees

We choose xgboost, you can see the principle of gradient boosted trees here: https://xgboost.readthedocs.io/en/stable/tutorials/model.html

In [ ]:
import xgboost as xgb

Separate training set and testing set, we use 80% of the data to train the model.

Label the testing dataset as `"test"`

In [ ]:
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
df_train, df_test = df[:train_size], df[train_size:len(df)].reset_index(drop=True)
df_test["data"] = "test"

Dependent variable: number of citations

Independent variable: number of years from the year of publication

In [ ]:
x_train_1 = df_train[["num_year_1"]]
y_train_1 = df_train["n_cits"]

x_test_1 = df_test[["num_year_1"]]
y_test_1 = df_test["n_cits"]

Define the model, see https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn for the complete list of parameters

In [ ]:
reg = xgb.XGBRegressor(tree_method="hist")

Fit the data by the model

In [ ]:
reg.fit(x_train_1, y_train_1)

Predict the citation by the model from the testing data set

Copy the testing dataframe frame, replace the actual number of citation by predicted number of citation, and label it as `"predicted"`

In [ ]:
y_pred_1 = reg.predict(x_test_1)
df_pred_1 = df_test.copy()
df_pred_1["n_cits"] = y_pred_1
df_pred_1["data"] = "predicted"

To analyze the performance of the model, we can compute the root mean square metric between the actual value and the predicted value

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
rms1 = sqrt(mean_squared_error(y_pred_1, y_test_1))
print("root mean square 1 = " + str(rms1))

We can also visualize the actual value and the predicted value as a function of the number of years since the year of publication.

In [ ]:
df_pred_test_1 = pd.concat([df_test, df_pred_1]).reset_index(drop=True)
ax = sns.lineplot(data=df_pred_test_1, x="num_year_1", y="n_cits", hue="data")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax

Let's try once more, but we add `"is_open_access"` as an addition independent variable.

In [ ]:
x_train_2 = df_train[["num_year_1", "is_open_access"]]
y_train_2 = df_train["n_cits"]

x_test_2 = df_test[["num_year_1", "is_open_access"]]
y_test_2 = df_test["n_cits"]

In [ ]:
reg.fit(x_train_2, y_train_2)

In [ ]:
y_pred_2 = reg.predict(x_test_2)
df_pred_2 = df_test.copy()
df_pred_2["n_cits"] = y_pred_2
df_pred_2["data"] = "predicted"

Adding extra inforation does not necessarily improve the predictive power

In [ ]:
rms2 = sqrt(mean_squared_error(y_pred_2, y_test_2))
print("root mean square 1 = " + str(rms1))
print("root mean square 2 = " + str(rms2))

In [ ]:
df_pred_test_2 = pd.concat([df_test, df_pred_2]).reset_index(drop=True)
ax = sns.lineplot(data=df_pred_test_2, x="num_year_1", y="n_cits", hue="data")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax

In [ ]:
df_pred_test_2.head()

### Save the dataframe back to the MSSQL server

You may want to save what you get from python back to the sql server.

Here, we create a new schema named `practical`, and save the dataframe `df_pred_test_2` as a database named `intro`

In [ ]:
import sqlalchemy as db

In [ ]:
if "practical" not in engine.dialect.get_schema_names(engine):
    engine.execute(db.schema.CreateSchema("practical"))
    
df_pred_test_2.to_sql("intro", con=engine, schema="practical", if_exists='replace')

And you can now see the database in the sql server, remember to change `laikh` to your username

In [ ]:
%%sql
SELECT TOP (5) *
FROM userdb_laikh.practical.intro